In [ ]:
import tensorflow as tf
import numpy      as np

import directoryFunctions
import pathlib
import data
import time
import os

from tensorflow.keras.applications        import InceptionV3
from tensorflow.keras.optimizers          import Adam
from tensorflow.keras.callbacks           import CSVLogger, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers              import Dense, GlobalAveragePooling2D
from tensorflow.keras.losses              import CategoricalCrossentropy
from tensorflow.keras.models              import load_model, Sequential ####

In [ ]:
def getFrameCount(dataDirectory):
    return len(list(dataDirectory.glob('*/*.jpg')))

In [ ]:
def getDataSet(dataDirectory):
    imagePathsDataset = tf.data.Dataset.list_files(str(dataDirectory/'*/*'))
    return imagePathsDataset

In [ ]:
def getLabeledData(imagePath):
    def getLabel(imagePath):
        # convert the path to a list of path components
        parts = tf.strings.split(imagePath, os.path.sep)
        # The second to last is the class-directory
        return parts[-2] == classNames
    
    def decodeImg(img):
        # convert the compressed string to a 3D uint8 tensor
        img = tf.image.decode_jpeg(img, channels=3)
        # Use `convert_image_dtype` to convert to floats in the [0,1] range.
        img = tf.image.convert_image_dtype(img, tf.float32)
        # resize the image to the desired size.
        return tf.image.resize(img, [299, 299])
    
    label = getLabel(imagePath)
    # load the raw data from the file as a string
    img = tf.io.read_file(imagePath)
    img = decodeImg(img)
    return img, label

In [ ]:
def prepareTrainDataset(dataset, cache, shuffleBufferSize):
    if cache:
        if isinstance(cache, str):
            dataset = dataset.cache(cache)
        else:
            dataset = dataset.cache()
    
    dataset = dataset.shuffle(buffer_size = shuffleBufferSize)
    # Repeat forever
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    # `prefetch` lets the dataset fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
def getModel():
    base_model = InceptionV3(input_shape = (299, 299, 3),
                             include_top = False,
                             weights     = 'imagenet')
    base_model.trainable = False
    global_average_layer = GlobalAveragePooling2D()
    prediction_layer     = Dense(numClasses, activation = 'softmax')
    
    model = Sequential([base_model, global_average_layer, prediction_layer])
    
    model.compile(optimizer = Adam(lr = 0.0001),
                  loss      = CategoricalCrossentropy(from_logits = True),
                  metrics   = ['accuracy'])
    return model

In [ ]:
def fineTuneModel(model, fineTuneAt):
    base_model = model.layers[0]
    base_model.trainable = True
    
    for layer in base_model.layers[:fineTuneAt]:
        layer.trainable = False
        
    model.compile(optimizer = Adam(lr = 0.00001),
                  loss      = CategoricalCrossentropy(from_logits = True),
                  metrics   = ['accuracy'])
    return model

In [ ]:
def trainModel(model, initial_epoch, epochs, trainDataset, validationDataset, steps_per_epoch, validation_steps, callbacks):
    history = model.fit(trainDataset,
                        initial_epoch    = initial_epoch, 
                        epochs           = epochs, 
                        validation_data  = validationDataset,
                        steps_per_epoch  = steps_per_epoch,
                        validation_steps = validation_steps,
                        callbacks        = callbacks)
    return model, history

In [ ]:
dataObj    = data.Data()
numClasses = dataObj.numClasses
classNames = np.array(dataObj.classes)

AUTOTUNE   = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [ ]:
def main():
    initial_epoch    = 0
    epochs           = 1
    fine_tune_epochs = 2
    fineTuneAt       = 249
    
    cachePath     = pathlib.Path(r"./Cache")
    cacheFilePath = cachePath/'trainCNNDatasetCache'
    directoryFunctions.removeDirectory(cachePath) ####
    directoryFunctions.createDirectory(cachePath)
    
    rootPath      = pathlib.Path(r"D:\ActionRecognition")
    framesPath    = rootPath/'Frames'
    
    trainDataDirectory       = framesPath/'Train'
    validationdataDirectory  = framesPath/'Validation'
    cnnCallbacksDirectory    = rootPath/'Callbacks'/'CNN'/f'{numClasses}'
    
    trainFrameCount      = getFrameCount(trainDataDirectory)
    validationFrameCount = getFrameCount(validationdataDirectory)
    
    trainDataset      = getDataSet(trainDataDirectory)
    validationDataset = getDataSet(validationdataDirectory)
    trainDataset      = trainDataset.map(getLabeledData,      num_parallel_calls=AUTOTUNE)
    validationDataset = validationDataset.map(getLabeledData, num_parallel_calls=AUTOTUNE)
    
    trainDataset      = prepareTrainDataset(trainDataset, str(cacheFilePath), trainFrameCount)
    validationDataset = validationDataset.batch(BATCH_SIZE)
    
    steps_per_epoch  = np.ceil(trainFrameCount/BATCH_SIZE)
    validation_steps = np.ceil(validationFrameCount/BATCH_SIZE)
    
    modelCheckpointDirectory = cnnCallbacksDirectory/'ModelCheckpoint'
    tensorboardDirectory     = cnnCallbacksDirectory/'Tensorboard'
    csvLoggerDirectory       = cnnCallbacksDirectory/'CSVLogger'
    
    directoryFunctions.createDirectory(csvLoggerDirectory)
    directoryFunctions.createDirectory(modelCheckpointDirectory) ####
    
    modelCheckpoint = ModelCheckpoint(filepath       = str(modelCheckpointDirectory/(f'{int(time.time())}' + '_CNN_{epoch:03d}_{val_loss:.2f}.h5')),
                                      save_best_only = True) #### add .h5 and added timeinfornt of it
    tensorboard     = TensorBoard(log_dir = str(tensorboardDirectory/f'{int(time.time())}'))
    csvLogger       = CSVLogger(str(csvLoggerDirectory/f'{int(time.time())}.log'))
    earlyStopping   = EarlyStopping(monitor = 'val_loss', patience = 5)
    callbacks       = [modelCheckpoint, tensorboard, csvLogger, earlyStopping]
    
    ####---------------
    savedModelPath = "" # insert path to saved model (.h5 file) here
    if savedModelPath == "":
        model                  = getModel()
        trained_model, history = trainModel(model, initial_epoch, epochs, 
                                            trainDataset, validationDataset, 
                                            steps_per_epoch, validation_steps, [])
        fine_tuned_model       = fineTuneModel(trained_model, fineTuneAt)
        initial_epoch          = history.epoch[-1]
    else:
        fine_tuned_model = load_model(savedModelPath)
        initial_epoch    = 0
        fine_tune_epochs = 1
    ####---------------
    
    trained_model, history_fine = trainModel(fine_tuned_model, 
                                             initial_epoch, fine_tune_epochs, 
                                             trainDataset, validationDataset,
                                             steps_per_epoch, validation_steps,
                                             callbacks) #### history.epoch[-1] -> initial_epoch
    
    directoryFunctions.removeDirectory(cachePath)

In [ ]:
main()